In [1]:
since_date = '2021-07-13'
until_date = '2021-07-14'

In [2]:
import os
import pandas as pd
# import snscrape.modules.twitter as sntwitter
import numpy as np
import datetime
# from capstone_twitter_search import twittsearch

import re                                        # to handle regular expressions
# from string import punctuation                   # to extract the puntuation symbols
# import nltk
from nltk.tokenize import word_tokenize          # to divide strings into tokens
# from nltk.stem import WordNetLemmatizer          # to lemmatize the tokens
from nltk.corpus import stopwords                # to remove the stopwords 

# import random                                    # for generating (pseudo-)random numbers
# import matplotlib.pyplot as plt                  # to plot some visualizations
# import datetime
import tensorflow as tf            
# from tensorflow import keras
# from tensorflow.keras import layers
# from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
# from tensorflow.keras.layers import Embedding
# from tensorflow.keras.callbacks import EarlyStopping
# from mordecai import Geoparser
# #from mordecai import batch_geoparse
# from flatten_json import flatten_json

# import json
# from tensorflow.keras import layers
import text_proccessing
from text_proccessing import clean_text
from text_proccessing import remove_stopwords
from text_proccessing import lemmatize_text
from text_proccessing import concatenate_text
from text_proccessing import makeglove
from text_proccessing import make_embedding_matrix

import modhelp
from modhelp import train_val_split
from modhelp import geo_df
from modhelp import suggest_nn2

In [10]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/dawnarbogast/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dawnarbogast/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/dawnarbogast/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [3]:
code_dir = os.getcwd()

parent_dir = os.path.dirname(code_dir)
data_dir = os.path.join(parent_dir,"Data")
dsa= os.path.dirname(parent_dir)
tweet_dir = os.path.join(parent_dir,"TweetMap")
directory = os.path.join(data_dir,'splits')


In [4]:
train_data = pd.read_csv(os.path.join(directory, 'InformativenessTrain_Tokenized.csv'),engine='python')
train_data['text']= train_data['text'].astype(str)

In [5]:
start =  datetime.datetime.now() 
print('splitthis started at: {}'.format(start))
train_samples, val_samples, train_labels, val_labels = train_val_split(train_data, 0.25)
end =  datetime.datetime.now() 
print ('Text done {}'.format(end-start))
start =  datetime.datetime.now() 
print('Loading model,embedding and glove commenced at  {}'.format(start))

model = tf.keras.models.load_model(os.path.join(dsa,'model1'))
print("loading vecotor")
path_to_glove_file = os.path.join(dsa,'WordVector','glove.twitter.27B.200d.txt')

print("indexing")
embeddings_index=makeglove(path_to_glove_file)
print("matrix and vectorize")
embedding_matrix, vectorizer = make_embedding_matrix(train_samples, val_samples, embeddings_index)
end =  datetime.datetime.now() 

print("embedded in {}".format(end-start))

splitthis started at: 2021-07-16 18:30:01.522683
Total size of the dataset: 117339.
Training dataset: 88005.
Validation dataset: 29334.
Text done 0:00:00.021193
Loading model,embedding and glove commenced at  2021-07-16 18:30:01.544074
loading vecotor
indexing
matrix and vectorize
Converted 32304 words (22696 misses).
embedded in 0:01:55.306570


In [7]:
twts = pd.read_csv(os.path.join(tweet_dir,'tweets_df_{0}_{1}.csv'.
                              format(since_date.replace('-',''),until_date.replace('-',''))), engine = 'python')
twts['Text'] = twts['Text'].astype(str)
twts = twts[['Datetime', 'TweetId', 'Text','TweetLink']]

In [11]:
procstart =  datetime.datetime.now()
print("{}: processing ".format(procstart))
twts['ptext'] = twts['Text'].apply(lambda x: clean_text(x))
twts['ptext'] = twts['ptext'].apply(lambda x: word_tokenize(x))
twts['ptext'] = twts['ptext'].apply(lambda x : remove_stopwords(x))
twts['ptext'] = twts['ptext'].apply(lambda x : lemmatize_text(x))
twts['ptext'] = twts['ptext'].apply(lambda x : concatenate_text(x))
procend=  datetime.datetime.now() 
print("{}: processed in {}".format(procend, procend - procstart))



2021-07-16 18:38:14.912106: processing 
2021-07-16 18:40:27.817121: processed in 0:02:12.905015


In [12]:

predstart = datetime.datetime.now() 
print("{}: predict".format(predstart))
predictions = suggest_nn2(twts, model,vectorizer)

submission_data = {"ID": twts['TweetId'].tolist(),"tweet": twts['Text'].tolist(), "target": predictions}

submission_df = pd.DataFrame(submission_data)
result = twts.join(submission_df.target)
predend = datetime.datetime.now() 
print("predicted {}".format(predend-predstart))
# 
result_inf = result[result['target']==0]
print("working over {} informative tweets".format(len(result_inf)))



2021-07-16 18:40:27.847588: predict
predicted 0:00:30.102334
working over 25581 informative tweets


In [ ]:


print("Run geoprocessing over tweets")

geostart =  datetime.datetime.now() 
print("{}: geostart".format(geostart))
df_js = geo_df(result_inf)
geoend =  datetime.datetime.now()
print("{}: geoend {}".format(geoend,geoend-geostart))

df_js.to_csv(os.path.join(tweet_dir,'result_{0}_{1}.csv'.
                              format(since_date.replace('-',''),until_date.replace('-',''))))
print("fín")

Run geoprocessing over tweets
2021-07-16 19:29:40.002891: geostart
Models path: /Users/dawnarbogast/opt/anaconda3/lib/python3.8/site-packages/mordecai/models/
You may be using an outdated Geonames index/Mordecai version.
Your index is from 2018-06-05, while your Mordecai version is from 2020-07-11. Please see
https://github.com/openeventdata/mordecai/ for instructions on updating.


  0%|          | 46/25581 [00:06<44:52,  9.48it/s]  Traceback (most recent call last):
  File "/Users/dawnarbogast/opt/anaconda3/lib/python3.8/site-packages/mordecai/geoparse.py", line 726, in infer_country
    prediction = self.country_model.predict(i['matrix']).transpose()[0]
  File "/Users/dawnarbogast/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py", line 1727, in predict
    tmp_batch_outputs = self.predict_function(iterator)
  File "/Users/dawnarbogast/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/eager/def_function.py", line 889, in __call__
    result = self._call(*args, **kwds)
  File "/Users/dawnarbogast/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/eager/def_function.py", line 924, in _call
    results = self._stateful_fn(*args, **kwds)
  File "/Users/dawnarbogast/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/eager/function.py", line 3022, in __call__
    filtered_flat_args) = self._maybe_define_f

None


  1%|          | 161/25581 [00:19<24:45, 17.11it/s]  Traceback (most recent call last):
  File "/Users/dawnarbogast/opt/anaconda3/lib/python3.8/site-packages/mordecai/geoparse.py", line 726, in infer_country
    prediction = self.country_model.predict(i['matrix']).transpose()[0]
  File "/Users/dawnarbogast/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py", line 1727, in predict
    tmp_batch_outputs = self.predict_function(iterator)
  File "/Users/dawnarbogast/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/eager/def_function.py", line 889, in __call__
    result = self._call(*args, **kwds)
  File "/Users/dawnarbogast/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/eager/def_function.py", line 924, in _call
    results = self._stateful_fn(*args, **kwds)
  File "/Users/dawnarbogast/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/eager/function.py", line 3022, in __call__
    filtered_flat_args) = self._maybe_define_

None


...

 27%|██▋       | 6939/25581 [13:34<15:52, 19.58it/s]Traceback (most recent call last):
  File "/Users/dawnarbogast/opt/anaconda3/lib/python3.8/site-packages/mordecai/geoparse.py", line 726, in infer_country
    prediction = self.country_model.predict(i['matrix']).transpose()[0]
  File "/Users/dawnarbogast/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py", line 1727, in predict
    tmp_batch_outputs = self.predict_function(iterator)
  File "/Users/dawnarbogast/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/eager/def_function.py", line 889, in __call__
    result = self._call(*args, **kwds)
  File "/Users/dawnarbogast/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/eager/def_function.py", line 924, in _call
    results = self._stateful_fn(*args, **kwds)
  File "/Users/dawnarbogast/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/eager/function.py", line 3022, in __call__
    filtered_flat_args) = self._maybe_define_f

None
None


 28%|██▊       | 7058/25581 [13:43<25:45, 11.99it/s]Traceback (most recent call last):
  File "/Users/dawnarbogast/opt/anaconda3/lib/python3.8/site-packages/mordecai/geoparse.py", line 726, in infer_country
    prediction = self.country_model.predict(i['matrix']).transpose()[0]
  File "/Users/dawnarbogast/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py", line 1727, in predict
    tmp_batch_outputs = self.predict_function(iterator)
  File "/Users/dawnarbogast/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/eager/def_function.py", line 889, in __call__
    result = self._call(*args, **kwds)
  File "/Users/dawnarbogast/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/eager/def_function.py", line 924, in _call
    results = self._stateful_fn(*args, **kwds)
  File "/Users/dawnarbogast/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/eager/function.py", line 3022, in __call__
    filtered_flat_args) = self._maybe_define_f

None


 28%|██▊       | 7097/25581 [13:46<25:02, 12.31it/s]

In [ ]:
df_js.head()